In [34]:
import nltk
import requests
import warnings
from sqlalchemy import text
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer

import sqlite3

# Data manipulation and analysis
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sqlalchemy.orm import sessionmaker
import matplotlib.dates as mdates

from datetime import datetime
from datetime import timedelta

# Financial data extraction
import yfinance as yf

# SQL Alchemy for database interaction
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session

# Additional libraries for analysis or visualization
import matplotlib.pyplot as plt
import seaborn as sns
from config import api_key

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [35]:
conn = sqlite3.connect('stock_market_analysis.sqlite')
db = conn.cursor()

In [36]:
engine = create_engine('sqlite:///stock_market_analysis.sqlite')
session = Session(engine)


In [37]:
inspect(engine).get_table_names()

['Average_Sentiment_Score', 'stock_history']

In [38]:
pd.read_sql('SELECT * FROM Average_Sentiment_Score', engine)

,index,Ticker,Sentiment Score
0,0,AAPL,0.040923
1,1,GOOGL,0.041079
2,2,MSFT,0.026950


In [39]:
pd.read_sql('SELECT * FROM stock_history', engine)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2020-01-02 00:00:00.000000,74.059998,75.150002,73.797501,75.087502,72.960472,135480400,AAPL
1,2020-01-03 00:00:00.000000,74.287498,75.144997,74.125000,74.357498,72.251137,146322800,AAPL
2,2020-01-06 00:00:00.000000,73.447502,74.989998,73.187500,74.949997,72.826843,118387200,AAPL
3,2020-01-07 00:00:00.000000,74.959999,75.224998,74.370003,74.597504,72.484337,108872000,AAPL
4,2020-01-08 00:00:00.000000,74.290001,76.110001,74.290001,75.797501,73.650352,132079200,AAPL
...,...,...,...,...,...,...,...,...
2263,2022-12-23 00:00:00.000000,236.110001,238.869995,233.940002,238.729996,235.769760,21207000,MSFT
2264,2022-12-27 00:00:00.000000,238.699997,238.929993,235.830002,236.960007,234.021713,16688600,MSFT
2265,2022-12-28 00:00:00.000000,236.889999,239.720001,234.169998,234.529999,231.621841,17457100,MSFT
2266,2022-12-29 00:00:00.000000,235.649994,241.919998,235.649994,241.009995,238.021500,19770700,MSFT


In [40]:
tickers = ['TSLA', 'MSFT', 'IBM']
start='2023-04-01'
end='2024-04-01'

In [41]:
engine = create_engine('sqlite:///stock_market_analysis.sqlite')

stock_data = {ticker: yf.download(ticker, start, end) for ticker in tickers}
for ticker, df in stock_data.items():
    df['Ticker'] = ticker

combined_df = pd.concat(stock_data.values())
combined_df.to_sql('stock_history', con=engine, if_exists='replace', index=True)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


747

In [42]:
pd.read_sql('SELECT * FROM stock_history', engine)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2023-04-03 00:00:00.000000,199.910004,202.690002,192.199997,194.770004,194.770004,169545900,TSLA
1,2023-04-04 00:00:00.000000,197.320007,198.740005,190.320007,192.580002,192.580002,126463800,TSLA
2,2023-04-05 00:00:00.000000,190.520004,190.679993,183.759995,185.520004,185.520004,133882500,TSLA
3,2023-04-06 00:00:00.000000,183.080002,186.389999,179.740005,185.059998,185.059998,123857900,TSLA
4,2023-04-10 00:00:00.000000,179.940002,185.100006,176.110001,184.509995,184.509995,142154600,TSLA
...,...,...,...,...,...,...,...,...
742,2024-03-22 00:00:00.000000,192.000000,192.990005,190.509995,190.839996,188.964172,3987700,IBM
743,2024-03-25 00:00:00.000000,190.259995,190.820007,188.750000,188.789993,186.934326,3718300,IBM
744,2024-03-26 00:00:00.000000,189.020004,190.000000,188.500000,188.500000,186.647186,4229500,IBM
745,2024-03-27 00:00:00.000000,189.600006,190.960007,188.600006,190.800003,188.924576,3693300,IBM


In [43]:
stock_data.values()

dict_values([                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2023-04-03  199.910004  202.690002  192.199997  194.770004  194.770004   
2023-04-04  197.320007  198.740005  190.320007  192.580002  192.580002   
2023-04-05  190.520004  190.679993  183.759995  185.520004  185.520004   
2023-04-06  183.080002  186.389999  179.740005  185.059998  185.059998   
2023-04-10  179.940002  185.100006  176.110001  184.509995  184.509995   
...                ...         ...         ...         ...         ...   
2024-03-22  166.690002  171.199997  166.300003  170.830002  170.830002   
2024-03-25  168.759995  175.240005  168.729996  172.630005  172.630005   
2024-03-26  178.580002  184.250000  177.380005  177.669998  177.669998   
2024-03-27  181.410004  181.910004  176.000000  179.830002  179.830002   
2024-03-28  177.449997  179.570007  175.300003  175.789993  175.789993   

               Volume Ti

In [44]:
get_avg_sentiment_scores(tickers)

In [45]:
def get_avg_sentiment_scores(tickers):
    textual_data = {}

    for ticker in tickers:
        headlines = fetch_news(api_key, ticker)
        textual_data[ticker] = headlines

    # Analyzing sentiment for each headline and averaging the scores
    average_sentiment_scores = {}

    for ticker, headlines in textual_data.items():
        sentiments = [TextBlob(headline).sentiment.polarity if headline else 0 for headline in headlines]
        average_sentiment_scores[ticker] = np.mean(sentiments)

    # Extract tickers and sentiment scores
    tickers = list(average_sentiment_scores.keys())
    sentiment_scores = list(average_sentiment_scores.values())

    df = pd.DataFrame({"Ticker": tickers, "Sentiment Score":sentiment_scores})
    load_data_to_db(df, "Average_Sentiment_Score")

In [46]:
def fetch_news(api_key, ticker):
    base_url = "https://newsapi.org/v2/everything"
    params = {
        'q': ticker,             # Search query (ticker symbol)
        'sortBy': 'publishedAt', # Sort by publication date
        'apiKey': api_key        # Your NewsAPI key
    }
    response = requests.get(base_url, params=params)
    articles = response.json().get('articles', [])
    headlines = [article['title'] for article in articles]
    return headlines

In [47]:
get_avg_sentiment_scores(tickers)

In [48]:
def load_data_to_db(data, ticker):
    """
    Loads transformed data into the SQLite database.
    """
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    data.to_sql(ticker, con=engine, if_exists='replace', index=True)


In [49]:
# Reset the index and drop the old index
df.reset_index(drop=True, inplace=True)
print(df.head())  # Display the first few rows of the DataFrame
df


         Open        High         Low       Close   Adj Close   Volume Ticker
0  130.970001  132.610001  130.770004  132.059998  124.965782  3840100    IBM
1  131.990005  132.149994  130.889999  131.600006  124.530495  3382800    IBM
2  131.369995  132.610001  131.369995  132.139999  125.041489  2898700    IBM
3  132.160004  132.600006  130.320007  130.500000  123.489601  3050600    IBM
4  129.830002  131.080002  129.240005  131.029999  123.991112  2614400    IBM


,Open,High,Low,Close,Adj Close,Volume,Ticker
0,130.970001,132.610001,130.770004,132.059998,124.965782,3840100,IBM
1,131.990005,132.149994,130.889999,131.600006,124.530495,3382800,IBM
2,131.369995,132.610001,131.369995,132.139999,125.041489,2898700,IBM
3,132.160004,132.600006,130.320007,130.500000,123.489601,3050600,IBM
4,129.830002,131.080002,129.240005,131.029999,123.991112,2614400,IBM
...,...,...,...,...,...,...,...
244,192.000000,192.990005,190.509995,190.839996,188.964172,3987700,IBM
245,190.259995,190.820007,188.750000,188.789993,186.934326,3718300,IBM
246,189.020004,190.000000,188.500000,188.500000,186.647186,4229500,IBM
247,189.600006,190.960007,188.600006,190.800003,188.924576,3693300,IBM


In [50]:
def fetch_and_load_stock_data(tickers, start, end):
    stock_data = {}
    for ticker in tqdm(tickers, desc="Fetching data"):
        logger.info(f"Fetching data for {ticker}")
        try:
            data = yf.download(ticker, start=start, end=end)
            if not data.empty:
                data['Ticker'] = ticker
                stock_data[ticker] = data
                logger.info(f"Data for {ticker} fetched successfully.")
            else:
                logger.warning(f"No data found for {ticker}.")
        except Exception as e:
            logger.error(f"Failed to fetch data for {ticker}: {e}")

    if stock_data:
        df = pd.concat(stock_data.values())
        df.reset_index(inplace=True)
        load_data_to_db(df, 'stock_history')
        logger.info("Data loaded to the database successfully.")
    else:
        logger.warning("No data fetched for any of the tickers.")


In [51]:
def fetch_and_load_stock_data(tickers, start, end):
    stock_data = {}
    for ticker in tickers:
        print(f"Fetching data for {ticker}")
        try:
            stock_data[ticker] = yf.download(ticker, start, end)
            stock_data[ticker]['Ticker'] = ticker
        except Exception as e:
            print(f"Failed to get data for ticker '{ticker}': {e}")
            continue
    
    if stock_data:
        df = pd.concat(stock_data.values())
        df.reset_index(inplace=True)
        load_data_to_db(df, 'stock_history')
    else:
        print("No data fetched for the provided tickers.")


In [52]:
# Define Flask route with unique endpoint name
@app.route('/api/v1.0/load_stock_data/<tickers>/<start>/<end>', endpoint='load_stock_data_v2')
def load_stock_data(tickers, start, end):
    tickers_list = tickers.strip('[]').replace('%22', '').split(',')
    tickers_list = [ticker.strip(' "\'') for ticker in tickers_list]
    fetch_and_load_stock_data(tickers_list, start, end)
    return '<h1>Data has been loaded to the Database</h1>'


NameError: name 'app' is not defined

In [ ]:
pd.DataFrame(session.execute(text('SELECT * FROM Average_Sentiment_Score')))

In [ ]:
pd.DataFrame(session.execute(text('SELECT * FROM stock_history')))

In [ ]:
from sqlalchemy import create_engine, text, Table, Column, Integer, String, Float, MetaData
from sqlalchemy.orm import sessionmaker
import pandas as pd

# Define the database URL
DATABASE_URL = "sqlite:///your_database.db"  # Replace with your actual database URL

# Create the database engine
engine = create_engine(DATABASE_URL)

# Create a configured "Session" class
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Define the metadata
metadata = MetaData()

# Define the tables
average_sentiment_score = Table('Average_Sentiment_Score', metadata,
    Column('id', Integer, primary_key=True),
    Column('Date', String),
    Column('Sentiment', Float)
)

stock_history = Table('stock_history', metadata,
    Column('id', Integer, primary_key=True),
    Column('Date', String),
    Column('Ticker', String),
    Column('Close', Float)
)

# Create the tables in the database
metadata.create_all(engine)

# Insert sample data into the tables
def insert_sample_data():
    with engine.connect() as conn:
        conn.execute(text("""
            INSERT INTO Average_Sentiment_Score (Date, Sentiment) VALUES
            ('2024-01-01', 0.5),
            ('2024-01-02', 0.6),
            ('2024-01-03', 0.7)
        """))
        conn.execute(text("""
            INSERT INTO stock_history (Date, Ticker, Close) VALUES
            ('2024-01-01', 'AAPL', 150.0),
            ('2024-01-02', 'AAPL', 152.0),
            ('2024-01-03', 'AAPL', 154.0)
        """))

# Call the function to insert sample data
insert_sample_data()

# Function to execute query and convert result to DataFrame
def query_to_dataframe(query):
    with SessionLocal() as session:
        result = session.execute(text(query)).fetchall()
        df = pd.DataFrame(result)
        if len(df.columns) > 0:
            df.columns = result[0].keys()  # Assign column names from query result
    return df

# Example queries
df_sentiment = query_to_dataframe('SELECT * FROM Average_Sentiment_Score')
df_stock_history = query_to_dataframe('SELECT * FROM stock_history')

# Adjust DataFrame for stock history
if 'Date' in df_stock_history.columns:
    df_stock_history['Date'] = df_stock_history['Date'].str.replace(r'\s.*', '', regex=True)
else:
    print("Column 'Date' not found in DataFrame")

# Display the dataframes (optional)
print(df_sentiment.head())
print(df_stock_history.head())


In [53]:
import yfinance as yf
import pandas as pd

def update_stock_data(tickers, start, end):
    stock_data = {ticker: yf.download(ticker, start, end) for ticker in tickers }
    
    for ticker, df in stock_data.items():
         df['Ticker'] = ticker

    combined_df = pd.concat(stock_data.values())

    load_data_to_db(combined_df, 'stock_history')

    return "<h1>SQLite Database was updated</h1>"

tickers = ['AAPL', 'GOOGL', 'MSFT']
start = '2020-01-01'
end = '2023-01-01'
update_stock_data(tickers, start, end)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


'<h1>SQLite Database was updated</h1>'

In [54]:
import yfinance as yf
import pandas as pd

def update_stock_data(tickers, start, end):
    stock_data = {ticker: yf.download(ticker, start, end) for ticker in tickers}
    
    for ticker, df in stock_data.items():
        df['Ticker'] = ticker
    
    combined_df = pd.concat(stock_data.values())
    
    load_data_to_db(combined_df, 'stock_history')
    
    return '<h1>SQLite Database was updated</h1>'

#Example usage:
tickers = ['AAPL', 'GOOGL', 'MSFT']
start = '2020-01-01'
end = '2023-01-01'
update_stock_data(tickers, start, end)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


'<h1>SQLite Database was updated</h1>'

In [55]:
def fetch_all_stock_data(tickers, start='2023-04-01', end='2024-04-01'):
    get_avg_sentiment_scores(tickers)

    stock_data = {ticker: yf.download(ticker, start, end) for ticker in tickers}
    for ticker, df in stock_data.items():
        df['Ticker'] = ticker

    combined_df = pd.concat(stock_data.values())

    print(combined_df)
    
    load_data_to_db(combined_df, 'stock_history')

    return '<h1>SQLite Database was updated</h1>'

In [56]:
fetch_all_stock_data(tickers,start,end)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2020-01-02   74.059998   75.150002   73.797501   75.087502   72.960472   
2020-01-03   74.287498   75.144997   74.125000   74.357498   72.251137   
2020-01-06   73.447502   74.989998   73.187500   74.949997   72.826843   
2020-01-07   74.959999   75.224998   74.370003   74.597504   72.484337   
2020-01-08   74.290001   76.110001   74.290001   75.797501   73.650352   
...                ...         ...         ...         ...         ...   
2022-12-23  236.110001  238.869995  233.940002  238.729996  235.769760   
2022-12-27  238.699997  238.929993  235.830002  236.960007  234.021713   
2022-12-28  236.889999  239.720001  234.169998  234.529999  231.621841   
2022-12-29  235.649994  241.919998  235.649994  241.009995  238.021500   
2022-12-30  238.210007  239.960007  236.660004  239.820007  236.846252   

               Volume Ticker  
Date  

'<h1>SQLite Database was updated</h1>'

In [57]:
@app.route('/')
def home():
    return render_template('index.html')


NameError: name 'app' is not defined

In [ ]:
import sqlite3
import pandas as pd

# Function to list all tables in the database
def list_tables(db_connection):
    cursor = db_connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    return [table[0] for table in tables]

# Function to fetch data from a specified table and convert to DataFrame
def fetch_data_from_db(table_name, db_connection):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql_query(query, db_connection)
    return df

# Function to create a test table if no tables exist
def create_test_table(db_connection):
    cursor = db_connection.cursor()
    cursor.execute("""
        CREATE TABLE test_table (
            id INTEGER PRIMARY KEY,
            Date TEXT,
            Value REAL
        )
    """)
    cursor.execute("""
        INSERT INTO test_table (Date, Value) VALUES
        ('2024-01-01', 100.0),
        ('2024-01-02', 110.0),
        ('2024-01-03', 120.0)
    """)
    db_connection.commit()

# Connect to your SQLite database
db_path = 'your_database.db'  # Make sure this path is correct
db_connection = sqlite3.connect(db_path)

# List all tables
tables = list_tables(db_connection)
print("Available tables:", tables)

# If no tables are found, create a test table
if not tables:
    print("No tables found. Creating a test table...")
    create_test_table(db_connection)
    tables = list_tables(db_connection)
    print("Available tables after creating test table:", tables)

# Use an existing table name from the list of tables
if tables:
    table_name = tables[0]  # Replace with the desired table name
    print(f"Fetching data from table: {table_name}")

    # Fetch the data into a DataFrame
    df = fetch_data_from_db(table_name, db_connection)

    # Now you can safely call reset_index and access the Date column
    df.reset_index(inplace=True)
    print(df.head())  # Display the first few rows of the DataFrame

    # If there is a Date column, you can print it
    if 'Date' in df.columns:
        print(df.Date)
    else:
        print("No 'Date' column found in the DataFrame")
else:
    print("No tables found in the database.")


In [ ]:
import requests
import numpy as np
import pandas as pd
import yfinance as yf
from config import api_key
from textblob import TextBlob
from datetime import timedelta, datetime
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

def get_avg_sentiment_scores(tickers):
    textual_data = {}

    for ticker in tickers:
        headlines = fetch_news(api_key, ticker)
        textual_data[ticker] = headlines

    # Analyzing sentiment for each headline and averaging the scores
    average_sentiment_scores = {}

    for ticker, headlines in textual_data.items():
        sentiments = [TextBlob(headline).sentiment.polarity if headline else 0 for headline in headlines]
        average_sentiment_scores[ticker] = np.mean(sentiments)

    # Extract tickers and sentiment scores
    tickers = list(average_sentiment_scores.keys())
    sentiment_scores = list(average_sentiment_scores.values())

    df = pd.DataFrame({"Ticker": tickers, "Sentiment Score": sentiment_scores})
    load_data_to_db(df, "Average_Sentiment_Score")

def fetch_all_stock_data(tickers, start='2023-04-01', end='2024-04-01'):
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')

    stock_data = {ticker: yf.download(ticker, start, end) for ticker in tickers}
    
    print('Stock Data: ', stock_data)

    for ticker, df in stock_data.items():
        df['Ticker'] = ticker

    combined_df = pd.concat(stock_data.values())

    print('Data to stock_history: ', combined_df)

    combined_df.to_sql('stock_history', con=engine, if_exists='replace', index=True)
    
    get_avg_sentiment_scores(tickers)

    return '<h1>SQLite Database was updated</h1>'

# Function to fetch headlines from NewsAPI
def fetch_news(api_key, ticker):
    base_url = "https://newsapi.org/v2/everything"
    params = {
        'q': ticker,             # Search query (ticker symbol)
        'sortBy': 'publishedAt', # Sort by publication date
        'apiKey': api_key        # Your NewsAPI key
    }
    response = requests.get(base_url, params=params)
    articles = response.json().get('articles', [])
    headlines = [article['title'] for article in articles]
    return headlines

def load_data_to_db(data, ticker):
    """
    Loads transformed data into the SQLite database.
    """
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    data.to_sql(ticker, con=engine, if_exists='replace', index=True)

def read_data(ticker):
    """
    Reads the data for the specified ticker from the SQLite database.
    """
    engine = create_engine('sqlite:///stock_market_analysis.sqlite')
    query = f"SELECT * FROM '{ticker}'"
    data = pd.read_sql_query(query, con=engine)
    return data

def calculate_daily_return(stock_data):
    """
    Calculates the daily return from the closing prices.
    """
    stock_data['Daily_Return'] = stock_data['Close'].pct_change()
    return stock_data

def plot_moving_averages(stock_data, ticker):
    data = stock_data[ticker]
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()
    
    plt.figure(figsize=(14, 7))
    plt.plot(data['Close'], label='Close Price', alpha=0.5)
    plt.plot(data['SMA_50'], label='50-Day SMA')
    plt.plot(data['SMA_200'], label='200-Day SMA')
    plt.title(f"{ticker} - Moving Averages")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.show()

def predict_monthly_prices(model, last_known_price, last_known_date, months=36):
    future_prices = []
    future_dates = []
    current_price = last_known_price
    
    for month in range(months):
        # Approximating each month by 21 trading days
        for day in range(21):
            next_input = pd.DataFrame(data=[[current_price]], columns=['Previous Close'])
            current_price = model.predict(next_input)[0]
        
        future_prices.append(current_price)
        # Assuming last_known_date is a datetime object; add roughly 30 days for each month
        last_known_date += timedelta(days=30)
        future_dates.append(last_known_date)
    
    return future_dates, future_prices

# New function to plot predicted future prices
def plot_predict_future_prices(stock_data, ticker, months=12):
    data = stock_data[ticker].copy()
    
    # Prepare data for the model
    data['Date'] = pd.to_datetime(data.index)
    data['Date_ordinal'] = data['Date'].map(datetime.toordinal)
    
    # Train the model
    X = np.array(data['Date_ordinal']).reshape(-1, 1)
    y = data['Close'].values
    model = LinearRegression()
    model.fit(X, y)
    
    # Predict future prices
    last_date = data['Date'].iloc[-1]
    future_dates = [last_date + timedelta(days=i * 30) for i in range(1, months + 1)]
    future_dates_ordinal = np.array([date.toordinal() for date in future_dates]).reshape(-1, 1)
    future_prices = model.predict(future_dates_ordinal)
    
    # Plotting
    plt.figure(figsize=(14, 7))
    plt.plot(data['Date'], data['Close'], label='Historical Close Price', alpha=0.5)
    plt.plot(future_dates, future_prices, label='Predicted Future Prices', color='red', linestyle='--')
    plt.title(f"{ticker} - Future Price Prediction")
    plt.xlabel("Date")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.show()

# Objective Function (Negative Sharpe Ratio)
def neg_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate=0.01):
    p_var = np.dot(weights.T, np.dot(cov_matrix, weights))
    p_ret = np.dot(weights, expected_returns)
    return -(p_ret - risk_free_rate) / np.sqrt(p_var)

def analyze_sentiment(text):
    """Analyze the sentiment of a text and return polarity and subjectivity."""
    analysis = TextBlob(text)
    return analysis.sentiment.polarity, analysis.sentiment.subjectivity

def main():
    tickers = ['MSFT', 'CRM', 'CRWD', 'ZM', 'SHOP', 'AAPL']

    for ticker in tickers:
        print(f"Fetching news for: {ticker}")
        articles = fetch_news(api_key, ticker)
        if articles['status'] == 'ok':
            for article in articles['articles'][:5]:  # Show only the first 5 articles for brevity
                description = article.get('description') or article.get('title')
                polarity, subjectivity = analyze_sentiment(description)
                print(f"Ticker: {ticker}, Title: {article['title']}")
                print(f"Sentiment Polarity: {polarity:.2f}, Sentiment Subjectivity: {subjectivity:.2f}")
                print(f"URL: {article['url']}\n")
    
    return articles

def fetch_data(ticker):
    stock = yf.Ticker(ticker)
    data = stock.history(period="5y")
    return
